In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression # <--- Importamos a Regressão Logística
from sklearn.preprocessing import StandardScaler  # Essencial para LR
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)
import matplotlib.pyplot as plt

# ==============================================================================
# --- 1. Carregando e Pré-processando os Dados ---
# ==============================================================================
print("--- 1. Carregando e Pré-processando os Dados ---")

# ... (Seu código de carregamento e pré-processamento) ...
# (Ocultado por brevidade, mas é o mesmo código que você já tem)
# ... (Seu código de carregamento e pré-processamento) ...

# Carregar o dataset
file_path = '../data/raw/leish_dataset.csv'
df = pd.read_csv(file_path)

# Criar cópia para processamento
df_processed = df.copy()

# Lidar com valores ausentes (Missing)
for col in df_processed.select_dtypes(include=['object']).columns:
    df_processed[col] = df_processed[col].fillna('Unknown')

# Codificar a variável Alvo (Target)
target_map = {'positivo': 1, 'negativo': 0, 'Unknown': 0}
df_processed['diagnosis'] = df_processed['diagnosis'].map(target_map).astype(int)

# Separar features (X) e alvo (y)
X_categorical = df_processed.drop('diagnosis', axis=1)
y = df_processed['diagnosis']

# Aplicar One-Hot Encoding nas features categóricas
X_numeric = pd.get_dummies(X_categorical, drop_first=True, dtype=int)
# Salvar os nomes das colunas para o bônus
feature_names = X_numeric.columns.tolist() 

print(f"--- Pré-processamento Concluído. {len(feature_names)} features criadas. ---")


# ==============================================================================
# --- 2. Dividindo os Dados (Treino e Teste) ---
# ==============================================================================
print("\n--- 2. Dividindo os Dados (Treino e Teste) ---")

X_train, X_test, y_train, y_test = train_test_split(
    X_numeric, 
    y, 
    test_size=0.3,
    random_state=42,
    stratify=y
)

print(f"Tamanho do conjunto de Treino: {X_train.shape[0]}")
print(f"Tamanho do conjunto de Teste:  {X_test.shape[0]}")


# ==============================================================================
# --- 3. Aplicando StandardScaler (Fundamental para LR) ---
# ==============================================================================
print("\n--- 3. Aplicando StandardScaler (Obrigatório para LR) ---")

# A Regressão Logística CONVERGE muito melhor com dados padronizados
scaler = StandardScaler()

# Ajustar o scaler APENAS nos dados de TREINO
X_train_scaled = scaler.fit_transform(X_train)

# Transformar os dados de TESTE com o mesmo scaler
X_test_scaled = scaler.transform(X_test)

print("--- Dados escalados com sucesso ---")


# ==============================================================================
# --- 4. Modelo 8: Treinando a Regressão Logística ---
# ==============================================================================
print("\n--- 4. Modelo 8: Treinando a Regressão Logística ---")

# Inicializar o modelo
# 'class_weight='balanced'': Nossa estratégia principal
# 'solver='liblinear'': Um bom otimizador para datasets pequenos
lr_classifier = LogisticRegression(
    class_weight='balanced', 
    random_state=42,
    solver='liblinear' 
)

# Treinar o modelo nos dados de treino ESCALADOS
lr_classifier.fit(X_train_scaled, y_train)

print("--- Treinamento do Modelo 8 Concluído ---")


# ==============================================================================
# --- 5. Avaliando o Modelo 8 (Regressão Logística) ---
# ==============================================================================
print("\n--- 5. Avaliando o Modelo 8 (Regressão Logística) ---")

# Fazer previsões no conjunto de TESTE (escalado)
y_pred_lr = lr_classifier.predict(X_test_scaled)

# Calcular a Acurácia
print(f"Acurácia (Modelo 8 - LR): {accuracy_score(y_test, y_pred_lr):.4f}")

# Exibir Relatório de Classificação
print("\nRelatório de Classificação (Modelo 8 - LR):")
print(classification_report(y_test, y_pred_lr, target_names=['negativo (0)', 'positivo (1)']))

# Exibir Matriz de Confusão
print("\nMatriz de Confusão (Modelo 8 - LR):")
cm_lr = confusion_matrix(y_test, y_pred_lr)
print(f"            [Prev. Neg] [Prev. Pos]")
print(f"[Real Neg]  {cm_lr[0][0]:>10} {cm_lr[0][1]:>10}")
print(f"[Real Pos]  {cm_lr[1][0]:>10} {cm_lr[1][1]:>10}")


# ==============================================================================
# --- 6. BÔNUS: Coeficientes do Modelo (Interpretabilidade) ---
# ==============================================================================
print("\n\n--- 6. BÔNUS: Coeficientes da Regressão Logística ---")

# Extrair os coeficientes (pesos) que o modelo aprendeu
coefficients = lr_classifier.coef_[0]

# Criar um DataFrame para visualizar os coeficientes
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient (Peso)': coefficients
})

# Ordenar pelos valores (positivos e negativos)
coef_df = coef_df.sort_values(by='Coefficient (Peso)', ascending=False)

print("\n--- Top 10 Features que MAIS INDICAM 'POSITIVO' (Leishmaniose) ---")
print(coef_df.head(10))

print("\n--- Top 10 Features que MAIS INDICAM 'NEGATIVO' (Saudável) ---")
print(coef_df.tail(10).sort_values(by='Coefficient (Peso)', ascending=True))

--- 1. Carregando e Pré-processando os Dados ---
--- Pré-processamento Concluído. 43 features criadas. ---

--- 2. Dividindo os Dados (Treino e Teste) ---
Tamanho do conjunto de Treino: 319
Tamanho do conjunto de Teste:  137

--- 3. Aplicando StandardScaler (Obrigatório para LR) ---
--- Dados escalados com sucesso ---

--- 4. Modelo 8: Treinando a Regressão Logística ---
--- Treinamento do Modelo 8 Concluído ---

--- 5. Avaliando o Modelo 8 (Regressão Logística) ---
Acurácia (Modelo 8 - LR): 0.6569

Relatório de Classificação (Modelo 8 - LR):
              precision    recall  f1-score   support

negativo (0)       0.78      0.71      0.74        96
positivo (1)       0.44      0.54      0.48        41

    accuracy                           0.66       137
   macro avg       0.61      0.62      0.61       137
weighted avg       0.68      0.66      0.67       137


Matriz de Confusão (Modelo 8 - LR):
            [Prev. Neg] [Prev. Pos]
[Real Neg]          68         28
[Real Pos]       